In [1]:
import brian2 as br
import numpy as np
import pandas as pd
import repro_functions as repro
import scipy.io as sio

In [5]:
fol = r'f:\desktop\Code\Reproducibility\model_ICx\SynchResults\data\iccl'
group = 100

dataset = sio.loadmat(fol + '\\SynchOut_ITD' + str(group) + '_001.mat')
spikes = dataset['spikesICx']

del fol, dataset

In [7]:
dur = 0.3
cw = 0.0001 * br.second
bins = len(np.arange(0 * br.second, dur * br.second + 0.05 * br.second + cw, cw)) - 1
try:
    neurons, reps, params = np.shape(spikes)
except:
    neurons, reps = np.shape(spikes)
    params = 1

In [8]:
hist = np.ndarray([int(neurons), int(params), bins])

for p in range(params):
    for n in range(neurons):
        sp = spikes[n, :, p]
        sp = [s/1000 for s in sp]
        try:
            hist[n, p, :] = repro.make_PSTH(np.concatenate(sp, 1), dur * br.second, cw, reps, 0 * br.second)
        except:
            print(str(p), str(n))
            
del p, n, sp, s

IndexError: too many indices for array

In [18]:
# variables
curve_time = 100

rep_out = pd.DataFrame(columns = ['neuron', 'params', 'fr', 'curve'])

for p in range(params):
    for n in range(neurons):
        try:
            t, SAC = repro.autocor(hist[n, p, :] * br.Hz, reps, T=curve_time*br.ms, bin=cw)
            SAC = SAC * br.Hz ** 2
            sp = np.mean(hist[n, p, :])
        except:
            print(str(p), str(n))
            arr = np.arange(-(curve_time * br.ms), (curve_time * br.ms) , cw)
            arr = np.delete(arr, 0)
            arr[:] = np.nan
            SAC = arr
            del arr
            sp = np.nan
            pass
        tmp = {'neuron': n, 'params': p, 'fr':sp, 'curve': SAC}
        rep_out = rep_out.append(tmp, ignore_index = True)
    
del p, n, t, SAC, sp, tmp

In [19]:
curve = [c for c in rep_out['curve']]

sio.savemat('repro_ITD' + str(group) + '_001_constant.mat', {'curve':curve, 'cw':cw, 'neuron':rep_out['neuron'].tolist(), 'params':rep_out['params'].tolist(), 'fr':rep_out['fr'].tolist(), 'reps':reps, 'dur':dur})

del curve, c

In [20]:
del bins, curve_time, cw, dur, group, neurons, params, reps

In [21]:
del hist, spikes

In [22]:
del rep_out